# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [77]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2025-04-28 14:05:55,764 | INFO     |: Logger demo_notebook.log already initialized.
2025-04-28 14:05:55,765 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [156]:
# Start the plotter
START_PLOTTER()

2025-04-28 14:31:58,702 | INFO     |: Creating New Plotter Process


In [162]:
STOP_PLOTTER()

2025-04-28 14:33:29,847 | INFO     |: Plotter is stopped


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [ ]:
import csv
import datetime
import asyncio


class RealRobot():
    """A class to interact with the real robot
    """

    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(
            str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")

        self.config_params = load_config_params(self.world_config)

        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        # self.ble = ble

        self.is_done_collecting = False
        self.results = []
        self.ble.start_notify(
            self.ble.uuid['RX_STRING'], self.notification_handler)

    def notification_handler(self, uuid, byte_array):
        result = self.ble.bytearray_to_string(byte_array)
        if result == "DONE":
            LOG.info("Done Notification Received")
            self.is_done_collecting = True
        else:
            LOG.info("Data Notification Received...processing")
            time, imu, pwm, tof = self.ble.bytearray_to_string(
                byte_array).split('|')
            self.results.append(
                [float(time), float(imu), int(pwm), float(tof)])

    def get_pose(self):
        """Get robot pose based on odometry

        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.

        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        self.ble.send_command(CMD.START_PID, "")
        LOG.info("Performing Observation Loop")
        while self.is_done_collecting == False:
            await asyncio.sleep(0.1)

        LOG.info("Collecting Data")
        self.ble.send_command(CMD.SEND_PID_DATA, "")


        await asyncio.sleep(5)

        LOG.info('Finished receiving data.')
        LOG.info(self.results)

        imu_angles_deg = np.array([i[1] for i in self.results])
        tof_readings = np.array([i[3] for i in self.results])

        trials = 1

        angle = imu_angles_deg.reshape(
            (18, trials)).mean(axis=1)[np.newaxis].T
        distance = tof_readings.reshape((18, trials)).mean(axis=1)[
            np.newaxis].T / 1000  # mm -> m

        # save to csv file
        angle = '(5,3)'
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"robot_data_{timestamp}_{angle}.csv"

        with open(filename, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            # Write header
            csvwriter.writerow(['Time (ms)', 'IMU', 'PWM', 'TOF'])
            # Write all data rows
            csvwriter.writerows(self.results)

        print(f"Data saved to {filename}")

        return distance, angle

In [84]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-04-28 14:06:41,240 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:81:04:26:a3:64


KeyboardInterrupt: 

In [159]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-28 14:32:18,119 | INFO     |:  | Number of observations per grid cell: 18
2025-04-28 14:32:18,122 | INFO     |:  | Precaching Views...
2025-04-28 14:32:19,222 | INFO     |:  | Precaching Time: 1.098 secs
2025-04-28 14:32:19,223 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-28 14:32:19,223 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [ ]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(1.524, 0.9144)
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-28 14:33:05,242 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-28 14:33:05,242 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[25789.0, 0.0, 0.0, 440.0], [26787.0, 17.0, 0.0, 444.0], [27019.0, 37.0, 0.0, 459.0], [27441.0, 58.0, 0.0, 583.0], [28307.0, 77.0, 0.0, 523.0], [29840.0, 97.0, 0.0, 419.0], [31596.0, 117.0, 0.0, 395.0], [32188.0, 137.0, 0.0, 460.0], [33053.0, 157.0, 0.0, 637.0], [33677.0, 177.0, 0.0, 1071.0], [36757.0, 197.0, 0.0, 2482.0], [37523.0, 217.0, 0.0, 3015.0], [39761.0, 237.0, 0.0, 708.0], [40741.0, 257.0, 0.0, 475.0], [41279.0, 277.0, 0.0, 430.0], [41667.0, 297.0, 0.0, 1721.0], [42198.0, 317.0, 0.0, 1075.0], [43346.0, 337.0, 0.0, 642.0]]
2025-04-28 14:33:05,243 | INFO     |: Update Step
2025-04-28 14:33:05,245 | INFO     |:      | Update Time: 0.001 secs
2025-04-28 14:33:05,245 | INFO     |: Bel index     : (np.int64(10), np.int64(6), np.int64(7)) with prob = 0.9999999
2025-04-28 14:33:05,246 | INFO     |: Bel_b

Traceback (most recent call last):
  File "/Users/akin/Desktop/Desktop/dev/fastrobots/sim/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/akin/Desktop/Desktop/dev/fastrobots/sim/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
                      ^^^^^^^^^^^^^
AttributeError: type object 'Qt' has no attribute 'Key_Escape'


In [54]:
cmdr.get_pose()

Exception: Simulator is not running